Lets train a keras model for segmentation and see its fps on CPU ! 

In [1]:
import os, logging

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow_hub").setLevel(logging.CRITICAL)


from utils import (data_path, Display, get_train_data, DisplayCallback, DataGenerator)
import tensorflow as tf
from common import lr_schedule, get_segmentation_model

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
from train_segmentation import train

In [19]:
big_model_path = "demodir/" 
saved_big_model_path = "demodir/saved_model/"
tflite_model_path = "demodir/quantized.tflite"
video_path = "../yogaposes/videos/2019-10-10-183635.webm"

In [4]:
model = train(big_model_path)

In [4]:
model.summary()

NameError: name 'model' is not defined

In [14]:
!du -sh demodir/saved_model/

4.0K	demodir/saved_model/


In [12]:
from compress import compress

In [16]:
compress(saved_big_model_path, tflite_model_path, img_size=128)

NameError: name 'tflite_model_path' is not defined

In [17]:
!du -sh demodir/quantized.tflite

du: cannot access 'demodir/quantized.tflite': No such file or directory


In [18]:
from segmentation_speed import compare_speed

In [ ]:
compare_speed(big_model_path, tflite_model_path, video_path):